#### Note To Self

As of now, when I run the tectonic earth simulations for too long, the resulting mountains get too big. Although I have made attempts at solving this issue in the past, I feel like erosion processes is what I should be using. I should modify the Gospl notebook so that a new gospl simulation is run after each tectonic earth simulation iteration, and see if the Gospl errosion algorithm prevents my mountains from growing too high.

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from scipy.interpolate import griddata
from EarthsAssistant import EarthAssist

In [ ]:
#Code to confirm that all code based on previous notebooks works as expected
if False:
    earth = Earth(
                startTime = 10,
                endTime = 0,
                deltaTime = 1,
                baseUplift = 2000,
                distTransRange = 1000000, 
                numToAverageOver = 10,
                earthRadius = 6378137.,
                useKilometres = False,
                useGospl = True)
    
    #Run earth simulation, run Gospl simulation and animate results
    earth.runTectonicSimulation()
    #gosplMan = GosplManager(earth, subdivisions=4)
    #gosplMan.createAllFilesAndRunSimulation()
    #gosplMan.animateGosplOutput()
    earth.animate(lookAtLonLat=[0, 0])

# Introduction

As of now, our earth simulation seems to look ok as it runs over shorter periods of time. However, various errors become obvious as we run the simulation for longer. The most prominent problem so far is that when two continents diverge, they don't leave ocean behind. Additionally, we don't have a mechanism for ocean floor formations. These are both phenomina related to tectonic plate divergence which we will work towards solving in this notebook.

This problem can be demonstrated clearly in the resulting animation bellow:

In [ ]:
%%HTML
<video width="800" height="608" controls>
  <source src="./NotebookFiles/DivergenceProblem.mp4" type="video/mp4">
</video>

``` python
#Code for generating the above video
if False:
    earth = Earth(
        startTime = 100,
        endTime = 0,
        deltaTime = 2,
        baseUplift = 2000,
        distTransRange = 1000000, 
        numToAverageOver = 10,
        earthRadius = 6378137.,
        useKilometres = False,
        useGospl = True),
        useDivergeLowering = False
    earth.runTectonicSimulation()
    earth.animate()
```

# Diverging Continents

In order to make diverging contents leave oceans behind, we need to lower vertices that are nearby diverging plate boundaries. In our code, we will refer to the amount of lowering as *divergeLowering*, which will depend on the following transfers:

- **Distance Transfer** - The distance that a vertex is to the closest diverging plate boundary. Vertices that are close to a plate boundary should be lowered more than vertices far away from the boundaries, and so we will pass the distances through a gaussian shaped function to calculate the distance contribution to diverge lowering. Given the mean $\mu$ and variance $\sigma$, our gaussian is defined by:

$$
Gaussian(x, \mu, \sigma) = exp \Big( \frac{- (x - \mu)^2}{\sigma} \Big)
$$

- **Height Transfer** - The current height of the vertex. Vertices that are on a higher elevation (on land) should be lowered more than vertices that have a lower elevation (in ocean). Therefore the height contribution will be calculated by passing the current heights of vertices through a sigmoid function. Given the centre $\mu$ and steepness $s$, our sigmoid is defined by:

$$
Sigmoid(x, \mu, s) = \frac{1}{1 + e^{- s (x - \mu)}}
$$

In the code bellow, we define these two functions and provide a simple plot of each. Note that the default parameters are chosen so that most of the function outputs varies at input values in the range $x \in [-1, 1]$.

In [ ]:
#Gaussian function used for profile of diverge lowering
def gaussian(x, mean=0, variance=0.25):
    return np.exp(-((x - mean)**2) / variance)

#Sigmoid function
def sigmoid(x, centre=-0.1, steepness=6):
    return 1 / (1 + np.exp(-(x - centre) * steepness))

#Plot the gaussian and sigmoid functions
x = np.arange(-2, 2, 0.01)
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
axs[0].plot(x, gaussian(x, mean=0, variance=0.25))
axs[0].title.set_text("Gaussian")
axs[0].set_xlabel('Distance Input')
axs[0].set_ylabel('Output')
axs[1].plot(x, sigmoid(x, centre=0, steepness=4))
axs[1].title.set_text("Sigmoid")
axs[1].set_xlabel('Height Input')
axs[1].set_ylabel('Output')

Now that we have define the transfer functions for diverge lowering, we need to get the appropriate heights and distance input values. Getting the heights input is easy, since all we have to do is read the current heights from *Earth.heights*. However, to get the distances from diverging plate boundaries, we first need to get the location of all diverging plate boundaries, which can be identified by having a negative collision speed (as discussed in notebook 2). The distances will then be calculated based on the boundary lines. Most of the code for this is already contained in the *PlateBoundaries.py* file.

The code bellow demonstrates how we will get our distances to diverging plate boundaries.

In [ ]:
#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(boundaries):
    divXYZ, divLinePoints = [], []
    for bound in boundaries.plateBoundaries:
        for i in range(bound.lineCentres.shape[0]):
            if bound.collisionSpeed[i] < 0:
                divXYZ.append(bound.lineCentres[i])
                divLinePoints.append(bound.linePoints[i])
    return np.array(divXYZ), np.array(divLinePoints)

#Get distance from vertices to diverging plate boundaries.
def getDistanceToDivergence(divXYZ, sphereXYZ, linePoints):
    distIds = cKDTree(divXYZ).query(sphereXYZ, k=1)[1]
    distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)
    return distToBound

#Create an earth object
earth = Earth(
    startTime = 10,
    endTime = 0,
    deltaTime = 2,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True,
    useDivergeLowering=False)

#Create an instance of the Boundaries class (this code is normally called by earth.runTectonicSimulation())
time = earth.startTime - earth.deltaTime
plateIds = earth.getPlateIdsAtTime(time)
rotations = earth.getRotations(plateIds, time)
boundaries = Boundaries(time, earth, plateIds, rotations)

#Run the newly defined functions
divXYZ, divLinePoints = getDivergingBoundaries(boundaries)
distToDivs = getDistanceToDivergence(divXYZ, earth.sphereXYZ, divLinePoints)

#Visualize the results
plotter = pv.PlotterITK()
earthMesh = earth.getEarthMesh()
earthMesh['distToDivs'] = distToDivs**0.25
boundaryLines = Boundaries.getBoundaryLines(boundaries.plateBoundaries)
plotter.add_mesh(earthMesh, scalars='distToDivs')
plotter.add_mesh(boundaryLines, color='r')
plotter.show()

Using the data that we have just calculated, getting the diverge lowering is simple, as shown bellow:

In [ ]:
#Define function for diverge lowering
def getDivergeLowering(distToDivs, heights, maxDistance=200000, minMaxHeights=8000):
    distanceTransfer = gaussian(distToDivs / maxDistance)
    heightTransfer = sigmoid(heights / minMaxHeights)
    return distanceTransfer * heightTransfer

#Get diverge lowering
divergeLowering = getDivergeLowering(distToDivs, earth.heights)

#Visualize results
plotter = pv.PlotterITK()
earthMesh['divergeLowering'] = divergeLowering
plotter.add_mesh(earthMesh, scalars='divergeLowering')
plotter.add_mesh(boundaryLines, color='r')
plotter.show()

Now that we have gotten the diverge lowering property, we want to apply it at each of earth's simulation step. In the code bellow, the function *doSimulationStep()* is similar to *earth.doSimulationStep()*, but with diverge lowering also implemented in it. We then provide an animation of the results.

In [ ]:
#Given an earth object, apply diverge lowering to it
def applyDivergeLowering(earth, baseLowering=2000):
    divXYZ, divLinePoints = getDivergingBoundaries(earth.boundaries)
    distToDivs = getDistanceToDivergence(divXYZ, earth.sphereXYZ, divLinePoints)
    earth.heights -= baseLowering * getDivergeLowering(distToDivs, earth.heights)

#Function that is similar to earth.doSimulationStep() but with added code from this notebook
def doSimulationStep(earth, time):
    earth.heights = np.copy(earth.heightHistory[-1])
    earth.timeHistory.append(time - earth.deltaTime)
    earth.setPlateData(time)
    earth.heights += earth.boundaries.getUplifts()
    applyDivergeLowering(earth) #New line of code to be added in earth.doSimulationStep()
    earth.movePlatesAndRemesh()
    earth.heightHistory.append(earth.heights)
    earth.createTectonicDisplacements()
    
#Create new earth object
earth = Earth(
    startTime = 20,
    endTime = 0,
    deltaTime = 1,
    baseUplift = 2000,
    distTransRange = 1000000, 
    numToAverageOver = 10,
    earthRadius = 6378137.,
    useKilometres = False,
    useGospl = True,
    useDivergeLowering=False)

#Lopp that is similar to earth.runTectonicSimulation()
for time in earth.simulationTimes[:-1]:
    print('Currently simulating at {} Millions years ago'.format(time), end='\r')
    doSimulationStep(earth, time)

earth.animate(lookAtLonLat=[0, 0])

In [ ]:
%%HTML
<video width="800" height="608" controls>
  <source src="./TectonicSimulation.mp4" type="video/mp4">
</video>

# Sea Floor Spreading

The depth of oceans is primarily determined by the age of the ocean floors. This corrolation is made clear by comparing the heightmap of ocean floor and age map of ocean floors as shown bellow. So in order to simulated the evolution of ocean floor spreading, we will also need to keep track of the ocean floors age.

Note that the colormap in the pyvista visualization bellow shows ocean depths within the range of $[-5500, -2500]$, which is the depth range at which ocean floor spreading takes place.

In [ ]:
#Create new earth object
earth = Earth(startTime=1, endTime=0, deltaTime=1)

#Create mesh objects
earthMesh = earth.getEarthMesh()
earth.setPlateData(earth.startTime)
boundMesh = earth.boundaries.getBoundaryMesh()

#Create scalar mesh attributes to visualize ocean floor heights
oceanFloorHeights = earth.heights
oceanFloorHeights[oceanFloorHeights>-2500] = -2500
oceanFloorHeights[oceanFloorHeights<-5500] = -5500
earthMesh['oceanFloorHeights'] = oceanFloorHeights

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='oceanFloorHeights')
plotter.add_mesh(boundMesh)
plotter.show()

<div>
<img src="NotebookFiles/OceanFloorAge.jpg" width="1000">
</div>

### Initial Age Estimates

Before we can keep track of the ocean floor's age as the simulation runs, we first need an initial estimate of the ocean floors age at the first iteration of the tectonic earth's simulation. Since we don't have any height data available of the ocean floors during the early years of our simulations, we will have to approximate it based on tectonic boundary locations and plate velocities data provided to us by *Gplates*.

To begin with, we create a function that gets the location and speed data of oceanic rift locations. Then, for each vertex on our sphere that corresponds to an ocean, it's distance $d$ to rift boundaries, and the speed of divergence $s$ of those nearest boundaries. This is done using similar methods as for subduction uplift. Since speed is generally defined by distance over time $s = \frac{d}{t}$, the age $t$ of an ocean floor can be estimated by:

$$
t = \frac{d}{s + \epsilon}
$$
where $\epsilon$ is a small constant used purely to avoid division by zero.


In [ ]:
#Get coordinates and line points of all diverging plate boundary locations
def getOceanicRifts(bounds):
    riftXYZ, riftLinePoints, riftSpeeds = [], [], []
    for bound in bounds.plateBoundaries:
        if bound.gpmlBoundType == 'gpml:MidOceanRidge':
            for i in range(bound.lineCentres.shape[0]):
                if bound.collisionSpeed[i] < 0:
                    riftXYZ.append(bound.lineCentres[i])
                    riftLinePoints.append(bound.linePoints[i])
                    riftSpeeds.append(bound.collisionSpeed[i])
    return np.array(riftXYZ), np.array(riftLinePoints), np.array(riftSpeeds)

#Get distance and speeds from the sphere's vertices to oceanic rifts
def getDistAndSpeedToRifts(earth, bounds):
    riftXYZ, riftLinePoints, riftSpeeds = getOceanicRifts(bounds)
    distIds = cKDTree(riftXYZ).query(earth.sphereXYZ, k=10)[1]
    distIds[distIds >= riftXYZ.shape[0]] = riftXYZ.shape[0]-1
    closestLinePoints = riftLinePoints[distIds]
    riftSpds = riftSpeeds[distIds]
    distToRift = Boundaries.getDistsToLinesSeg(earth.sphereXYZ, closestLinePoints[:, 0])
    riftSpds = np.mean(riftSpds, axis=1)
    return distToRift, riftSpds

#Approximate the initial ocean floor age based on distance and speed to oceanic rifts
def getOceanFloorAge(earth, bounds, ageMultiplier=2, avoidZeroDivision=10):
    distToRift, riftSpeeds = getDistAndSpeedToRifts(earth, bounds)
    age = -distToRift / (riftSpeeds + avoidZeroDivision)
    age[earth.heights>0] = np.max(age)
    return ageMultiplier * age
    
#Create new earth object
earth = Earth(startTime = 5, endTime = 0, deltaTime = 1)
earth.runTectonicSimulation()
earth.setPlateData(earth.startTime)
bounds = earth.boundaries

#Run the new functions for getting ocean floor age
oceanFloorAge = getOceanFloorAge(earth, bounds)

#Get mesh objects for plotting
earthMesh = earth.getEarthMesh()
boundaryMesh = bounds.getBoundaryMesh()
contour = earthMesh.contour([0], scalars='heights')
earthMesh['oceanFloorAge'] = oceanFloorAge

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='oceanFloorAge')
plotter.add_mesh(contour, color="peru", opacity=1.)
plotter.add_mesh(boundaryMesh)
plotter.show()

The depth of ocean floors will be based on the following equation:

$$
depth = h = min
\begin{cases}
    2500 + 350 \sqrt{t} \\
    5500
\end{cases}
$$

To visualize this, we define an function that for the above equation and visualize it's profile. Note that the maximum depth occours when the ocean floor is 75 million years old.

In [ ]:
def getOceanDepth(age, riftDepth=2500, ageConst=350, maxDepth=5500):
    depth = riftDepth + ageConst * np.abs(age)**0.5
    depth[depth>maxDepth] = maxDepth
    return - depth

age = np.arange(-100, 100, 0.1)
plt.figure(figsize=(16, 2))
plt.plot(age, getOceanDepth(age))
plt.show()

In the earlier years of our simulation, we won't have ocean topological data available, and so we will initiate the ocean floor depths based on the above function. To show that this equation is indeed a good approximate, we apply this equation to our estimates of the ocean floor age to get the following results.

In [ ]:
oceanDepth = getOceanDepth(oceanFloorAge)
oceanDepth[earth.heights>0] = np.max(oceanDepth)
flatEarthMesh = earth.getFlatEarthMesh()

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(flatEarthMesh, scalars=oceanDepth)
plotter.show()

To confirm that we do indeed have a good approximation of the ocean floor's age, we can compare the above visualization will the image bellow. Since our model produces a maximum ocean depth at an ocean age of 75 million years, the dark blue region in our plot above correspond to ocean ages of older than 75 million years, which corresponds to the green regions in the image bellow.

<div>
<img src="NotebookFiles/OceanFloorAge.jpg" width="1000">
</div>

### Tracking Ocean Floor Age As Plates Move

Once we have initialized the ocean floors and their corresponding age, we now wish to simulate the change in height over time. Taking the derivative of the above equation with respect to time for $t \in [0, 75]$ Mya, we get the change in height $dh$ given a change in time $dt$ as:

$$
d h = \frac{350}{2\sqrt{t}} dt
$$

Note that as time passes, the ocean floors will move along with the tectonic plates. So for the same reasons that we need to interpolate heights during the remesh, we will also need to interpolate ocean floor age during its remesh.

Since most of the code for interpolating the ocean floor age is similar to that of interpolating heights after moving tectonic plates, we will redefine the remesh algorithm in such a way so that we can more easily add more scalars (Eg. Heights or ocean floor age) to be remeshed later on.

As discussed in tutorial 1, we begin by moving the tectonic plates along the sphere, convert the sphere into cylinder to help identify regions of overlapping plates through clustering means. These clusters will now be stored as a class attribute so that we can re-use them to interpolate any scalar field that moves along with the plates (such as height, ocean age, crust thickness...). The code bellow does this clustering, and provides a visualization of the clusters.

In [ ]:
#Algorithm for moving plates and without remeshing the sphere
def movePlates(earth):
    movedEarthXYZ = earth.movePlates(earth.plateIds, earth.rotations)
    movedLonLat = EarthAssist.cartesianToPolarCoords(movedEarthXYZ)
    earth.movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)

#Create cylinder of earth such that vertices are all equally spaced apart
#Overriding plates can then be identified as vertices being relatively closer to each other
def createEarthCylinder(earth):
    phiRes = earth.phiResolution
    thetaRes = earth.thetaResolution
    movedLonLat = earth.movedLonLat
    northToSoutDist = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    cylinderRadius = thetaRes * northToSoutDist / (np.pi * phiRes * 2)
    cylinderXYZ = EarthAssist.cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    return cylinderXYZ, thetaRes   
    
#Set the parameters "earth.isCluster" and "earth.clusterPointsNeighboursId"
#These will represent regions of overriding/colliding plates, and will be used for interpolating scalars after a remesh
def setRemeshClusters(earth):
    cylinderXYZ, thetaRes = createEarthCylinder(earth)
    
    #Run the clustering algorithm
    threshHoldDist = earth.clusterThresholdProportion * 360 / thetaRes
    cluster = DBSCAN(eps=threshHoldDist, min_samples=earth.minClusterSize).fit(cylinderXYZ)
    earth.isCluster = (cluster.labels_ != -1)

    #Create KDTree to find nearest neighbours of each point in cluster
    pointsInClusterLonLat = cylinderXYZ[earth.isCluster]
    clusterKDTree = cKDTree(pointsInClusterLonLat).query(pointsInClusterLonLat, k=earth.numOfNeighbsForRemesh+1)
    earth.clusterPointsNeighboursId = clusterKDTree[1]

#Create new earth object
earth = Earth(startTime = 60, endTime = 0, deltaTime = 10, minClusterSize=2)
earth.oceanFloorAge = getOceanFloorAge(earth, earth.boundaries)

#Move plates and set remesh clusters
movePlates(earth)
setRemeshClusters(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
isCluster = earth.isCluster

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(pv.Sphere(radius=earth.earthRadius-200000))
plotter.add_points(movedEarthXYZ[isCluster], color='r')
plotter.add_points(movedEarthXYZ[isCluster==False], color='b')
plotter.show()

Now that we have stored these clusters as a class attribute, we can interpolate any scalar attributes by simply adding one lines of code in the *interpolateScalars()* function bellow for each scalar. Note that we could easily extend this code so that we can interpolate vector fields too, however so far I have not found a need to do so.

To help visualize results, we have provided an earth mesh with contour lines and mesh topology based on old heights, and mesh scalar colors corresponding to the new interpolated heights. We have attached some additional scalar attributes to the mesh, which can be visualized by selecting them in the drop down menu in the top left of the pyvista window.

Note that when instantiating the earth object, we selected a large *deltaTime* of 10 My to make plate movement very visible. In practice we will use a finer *deltaTime* to simulate finer temporal details.

In [ ]:
#Interpolate scalars after moving tectonic plates
def interpolateScalars(earth):
    setRemeshClusters(earth)
    newHeights = interpolateScalar(earth, np.copy(earth.heights))
    newFloorAge = interpolateScalar(earth, np.copy(earth.oceanFloorAge))
    newFloorAge += earth.deltaTime
    return newHeights, newFloorAge

#Function based on interpolateHeights() from earth's remesh algorithm
def interpolateScalar(earth, scalar):
    scalarForRemesh = prepareScalarsForRemesh(earth, scalar)
    movedLonLat = earth.movedLonLat
    newScalar = griddata(movedLonLat, scalarForRemesh, earth.lonLat)
    whereNAN = np.argwhere(np.isnan(newScalar))
    newScalar[whereNAN] = griddata(movedLonLat, scalarForRemesh, earth.lonLat[whereNAN], method='nearest')
    return newScalar
    
#To prepare scalars for interpolation, we set scalars of overlaping vertices to the maximum of their neighbours
def prepareScalarsForRemesh(earth, scalars):
    isCluster = earth.isCluster
    clusterPointsNeighboursId = earth.clusterPointsNeighboursId
    scalarsInCluster = scalars[isCluster]
    neighbourScalars = scalarsInCluster[clusterPointsNeighboursId[:, 1:]]
    scalars[isCluster] = np.max(neighbourScalars, axis=1)
    return scalars

#Run the new functions for getting ocean floor age
newHeights, newFloorAge = interpolateScalars(earth)

#Get mesh objects for plotting
bounds = earth.boundaries
boundaryMesh = bounds.getBoundaryMesh()
earthMesh = earth.getEarthMesh()
earthMesh['newHeights'] = newHeights
earthMesh['oceanDepth'] = oceanDepth
earthMesh['newFloorAge'] = newFloorAge
earthMesh['oceanFloorAge'] = earth.oceanFloorAge
contour = earthMesh.contour([0], scalars='heights')

#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='newHeights')
plotter.add_mesh(contour, color="peru", opacity=1.)
plotter.show()

### Updating Ocean Floors Over Time

As we can see bellow, for the early years (Eg. 200 Mya), the PaleoDems data set used to initiate our earth does not provide any information about ocean depths, and instead just considers the ocean to be flat. Therefore it is desirable to set the ocean depth based on our estimates of ocean age and the equation bellow (as discussed above).

$$
depth = h = min
\begin{cases}
    2500 + 350 \sqrt{t} \\
    5500
\end{cases}
$$

In [ ]:
#Create new earth object
earth = Earth(startTime=200, endTime=0, deltaTime=5, minClusterSize=2)
oceanDepth = getOceanDepth(earth.oceanFloorAge)
oceanDepth[earth.heights>0] = np.max(oceanDepth)
bounds = earth.boundaries

#Create mesh objects
earthMesh = earth.getEarthMesh()
earthMesh['oceanDepth'] = oceanDepth
boundaryMesh = bounds.getBoundaryMesh()

#Plot stuff
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='oceanDepth')
plotter.add_mesh(boundaryMesh)
plotter.show()

We apply our approximate ocean depth to the initial earths heights

In [ ]:
#Any value bellow the threshold depth will be set to our approximate ocean depth
def setApproximateOceanDepths(earth, thresholdDepth=-2000):
    heights = earth.heights
    isBellowThreshold = (heights <= thresholdDepth)
    oceanDepth = getOceanDepth(earth.oceanFloorAge)
    earth.heights[isBellowThreshold] = oceanDepth[isBellowThreshold]
    earth.heightHistory[-1] = earth.heights

#Run new function and create mesh objects
earth = Earth(startTime=200, endTime=0, deltaTime=5, minClusterSize=2)
setApproximateOceanDepths(earth)
earthMesh = earth.getEarthMesh()

#Plot stuff
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(boundaryMesh)
plotter.show()

We have added the interpolation of ocean age into the main source code of Earth

$$
d h = \frac{350}{2\sqrt{t}} dt
$$

In [ ]:
#Function to apply changes in ocean floor depths
def updateOceanFloors(earth, thresholdDepth=-2000, ageToStop=75):
    dt = earth.deltaTime
    heights = earth.heights
    floorAge = earth.oceanFloorAge
    isYoungEnough = (floorAge - dt < ageToStop)
    isBellowThreshold = (heights <= thresholdDepth)
    updateHere = isBellowThreshold * isYoungEnough
    heights[updateHere] += 350 * dt / (2 * floorAge[updateHere]**0.5)
    earth.heights = heights

#Same function as in earth.doSimulationStep() but with addition of update ocean floors
def doSimulationStep(earth, time):
    earth.heights = np.copy(earth.heightHistory[-1])
    earth.timeHistory.append(time - earth.deltaTime)
    earth.setPlateData(time)
    earth.heights += earth.boundaries.getUplifts()
    earth.heights += earth.boundaries.getDivergeLowering()
    earth.movePlatesAndRemesh()
    updateOceanFloors(earth) #Run new ocean floor function here
    earth.heightHistory.append(earth.heights)
    earth.createTectonicDisplacements()

#Run new function and create mesh objects
earth = Earth(startTime=200, endTime=140, deltaTime=5, minClusterSize=2)
setApproximateOceanDepths(earth)
    
for time in earth.simulationTimes[:-1]:
    print('Currently simulating at {} Millions years ago'.format(time), end='\r')
    doSimulationStep(earth, time)

earthMesh = earth.getEarthMesh()
earthMesh['floorAge'] = earth.oceanFloorAge
boundMesh = earth.boundaries.getBoundaryMesh()

#Plot stuff
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars='heights')
plotter.add_mesh(boundMesh)
plotter.show()

After simply applying the change in ocean floors equation and running the simulation for a bit, we get the above results. Two related problems are clearly visible:

- As ocean floors diverge, the newly generated floor should have an age of zero, but at the moment the floor's age is set to some later age due to interpolation.

- Similarly, the newly generated floor should have an initial depth of -2500 metres, which is the oceanic ridge depth. Again our current depth is interpolated based on it's neighbours.

To solve both of these issues, we need to identify the newly created ocean floors, and assign the age and depth value for the newly generated ocean floors.

From the plot bellow, we can see that after moving tectonic plates and before remeshing the sphere, there are large areas with a lack of vertices. These areas are where new ocean floors need to be generated.

In [ ]:
def getOceanDepth(age, riftDepth=2500, ageConst=350, maxDepth=5500):
    depth = riftDepth + ageConst * np.abs(age)**0.5
    depth[depth>maxDepth] = maxDepth
    return - depth

#Algorithm for moving plates and without remeshing the sphere
def movePlates(earth):
    movedEarthXYZ = earth.movePlates(earth.plateIds, earth.rotations)
    movedLonLat = EarthAssist.cartesianToPolarCoords(movedEarthXYZ)
    earth.movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)

#Get pyvista lines of oceanic ridges
def getRidgeMesh(bounds):
    ridgeXYZ, lineConnectivity, xyzCount = [], [], 0
    for bound in bounds.plateBoundaries:
        if bound.gpmlBoundType == 'gpml:MidOceanRidge':
            numOfPoints = bound.XYZ.shape[0]
            lineConnectivity.append(numOfPoints)
            lineID = np.arange(numOfPoints) + xyzCount
            for i in range(numOfPoints):
                ridgeXYZ.append(bound.XYZ[i])
                lineConnectivity.append(lineID[i])
            xyzCount += numOfPoints
    return pv.PolyData(np.array(ridgeXYZ), lines=lineConnectivity)
    
#Create new earth object and move plates
earth = Earth(startTime=20, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get movedMesh
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

earth.setPlateData(earth.startTime - earth.deltaTime)
newBounds = earth.boundaries
ridgeMesh = getRidgeMesh(newBounds)

plotter = pv.PlotterITK()
plotter.add_mesh(movedMesh, scalars=earth.heights)
plotter.add_mesh(movedMesh.extract_all_edges(), color='white', opacity=0.2)
plotter.add_mesh(ridgeMesh, color='cyan')
plotter.show()

To generate the data for the new ocean floors, we need to identify the above large areas of missing vertices, fill in those areas with new vertices, assign an ocean floor age and height to those new vertices, and use the new vertices in our remesh algorithm. 

We begin by identifying the large areas of missing vertices. This can be done by using the area of each cell on our mesh, and the aspect ratio of each cell on our mesh. Multiplying these two values will give us a property which we will call aspect-area, and any cell with an unusually large aspect area will be considered to belong to plate divergence. We then also check that these identified cells are also close enough to an mid oceanic ridge boundary as given to us by pygplates. Results are shown bellow.

Also worth mentioning is that I've recently discovered an even better way for getting the distance from points on a mesh to lines. The pyvista function *compute_implicit_distance()* returns the distance from mesh points to another mesh (but not lines). The pyvista function *tube()* can be used to convert lines into a mesh of tubes, which for our purposes in calculating distances, basically is a line assuming we specified a small enough radius for the tubes. So all we have to do now, is convert the lines into tubes, and calculate the distance from our mesh to the tubes.

Note that in the following code, we will be taking advantage of multiple pyvista mesh filter functions, and I have found that trying to plot meshes with too many filter functions applied to them can break the plotterITK() (which uses the itkwidgets backend and only affects our visualizations). If any of the plots break, replace the *plotterITK()* with *plotter()*, which uses the original pyvista rendering backend. Interacting with plots with *plotter()* will be laggy, but at least they don't crash. The developers of *pyvista* are currently working on a *pythreejs* backend, wich should solve these issues.

In [ ]:
#A usefull metric for identifying diverging cells is by finding cells
#With a large aspectArea (cell aspect ratio multiplied by cell area)
def getAspectArea(mesh):
    cellQuality = mesh.compute_cell_quality(quality_measure='area')
    earthArea = cellQuality.cell_arrays['CellQuality']
    cellQuality = mesh.compute_cell_quality(quality_measure='aspect_ratio')
    aspectRatio = cellQuality.cell_arrays['CellQuality']
    return  np.array(aspectRatio * earthArea)

#A better method for calculating distances to lines that I only just recently discovered.
def getDistFromMeshToLines(mesh, lines):
    tubes = lines.tube(radius=10000, capping=False, n_sides=3)
    mesh = mesh.copy().compute_implicit_distance(tubes, inplace=True).point_data_to_cell_data()
    return np.array(mesh.cell_arrays['implicit_distance'])

#Create new earth object and move plates
earth = Earth(startTime=20, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get movedMesh
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

#Get ridge mesh
earth.setPlateData(earth.startTime - earth.deltaTime)
newBounds = earth.boundaries
ridgeMesh = getRidgeMesh(newBounds)

#Get areas of data that will help identify diverging boundaries
movedMeshCopy = movedMesh.copy()
aspectArea = getAspectArea(movedMeshCopy)
distToRidge = getDistFromMeshToLines(movedMeshCopy, ridgeMesh)

#Create isDiverging for identifying diverging plate boundaries
aspectAreaIsLarge = (aspectArea > np.median(aspectArea) * 4.0)
isCloseToRidge = (distToRidge < 1000000.0)
isDiverging = aspectAreaIsLarge * isCloseToRidge
movedMesh['isDiverging'] = isDiverging.astype(float)

#Plot results so far
plotter = pv.PlotterITK()
plotter.add_mesh(movedMesh, scalars='isDiverging')
plotter.add_mesh(ridgeMesh, color='cyan')
plotter.add_mesh(movedMesh.extract_all_edges(), color='white', opacity=0.2)
plotter.show()

Now that we have identified the cells bellonging to diverging oceanic rifts, we create a new diverging mesh by extracting the diverging cells from the movedMesh. To fill in the missing vertices, we subdivide this new mesh.

We then need to assign age and heights to these new vertices. The new vertices on the edge of the diverging mesh should have an age of *earth.deltaTime*, and the new vertices close to the oceanic ridge should have an age of 0 years. Any vertex inbetween should have a intermediate value. To assign values like age and height to these new vertices, we will need to know the proportion of the distance from the ridge, versus the distance from the diverging mesh edge that a new vertex is. After getting the distance from mesh edge $d_e$ and distance to oceanic ridge $d_r$, and normalizing these distance to a value between 0 and 1, we can get this propotion $p \in [0, 1]$ using:

$$
p = \frac{d_r}{d_r + d_e}
$$

In the code bellow, we visuallise the subdivided extracted diverging mesh color coded by the proportion defined above.

In [ ]:
#Extract diverging cells and subdivide
divergingCells = movedMesh.extract_cells(np.argwhere(isDiverging))
divergingEdges = divergingCells.extract_feature_edges(non_manifold_edges=False, feature_edges=False, manifold_edges=False)
divCellsSubbed = pv.PolyData(divergingCells.points, divergingCells.cells)
divCellsSubbed = divCellsSubbed.subdivide(3)

#Calculate the distances and their proportions
distToEdge = getDistFromMeshToLines(divCellsSubbed, divergingEdges)
distToRidge = getDistFromMeshToLines(divCellsSubbed, ridgeMesh)
distToEdge = EarthAssist.normalizeArray(distToEdge)
distToRidge = EarthAssist.normalizeArray(distToRidge)
proportion = distToRidge / (distToEdge + distToRidge)

#Convert the cell data to point data, making it suitable for vertices
divCellsSubbed['proportion'] = proportion
divCellsSubbed = divCellsSubbed.cell_data_to_point_data()
proportion = divCellsSubbed['proportion']

#Display results
plotter = pv.PlotterITK()
plotter.add_mesh(divCellsSubbed, scalars=proportion)
plotter.add_mesh(divergingEdges, color='r')
plotter.add_mesh(ridgeMesh, color='cyan')
plotter.add_mesh(divCellsSubbed.extract_all_edges(), color='black', opacity=0.1)
plotter.show()

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from scipy.interpolate import griddata
from EarthsAssistant import EarthAssist

def getOceanDepth(age, riftDepth=2500, ageConst=350, maxDepth=5500):
    depth = riftDepth + ageConst * np.abs(age)**0.5
    depth[depth>maxDepth] = maxDepth
    return - depth

#Algorithm for moving plates and without remeshing the sphere
def movePlates(earth):
    movedEarthXYZ = earth.movePlates(earth.plateIds, earth.rotations)
    movedLonLat = EarthAssist.cartesianToPolarCoords(movedEarthXYZ)
    earth.movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)

In [ ]:
#Create new earth object and move plates
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

#Get aspectArea, a nice way of identifying diverging mesh areas
cellQuality = movedMesh.compute_cell_quality(quality_measure='area')
earthArea = cellQuality.cell_arrays['CellQuality']
cellQuality = movedMesh.compute_cell_quality(quality_measure='aspect_ratio')
aspectRatio = cellQuality.cell_arrays['CellQuality']
aspectArea = aspectRatio * earthArea

#Need distance to oceanic rifts
earth.setPlateData(earth.startTime - earth.deltaTime)
newBounds = earth.boundaries

xyzCount = 0
newBoundsXYZ, lineConnectivity = [], []
for bound in newBounds.plateBoundaries:
    if bound.gpmlBoundType == 'gpml:MidOceanRidge':
        numOfPoints = bound.XYZ.shape[0]
        lineConnectivity.append(numOfPoints)
        lineID = np.arange(numOfPoints) + xyzCount
        for i in range(numOfPoints):
            newBoundsXYZ.append(bound.XYZ[i])
            lineConnectivity.append(lineID[i])
        xyzCount += numOfPoints

#Get distance to ridge
ridgeMesh = pv.PolyData(np.array(newBoundsXYZ), lines=lineConnectivity)
ridgeTube = ridgeMesh.tube(radius=10000, capping=False, n_sides=3)
movedMesh = movedMesh.compute_implicit_distance(ridgeTube, inplace=True).point_data_to_cell_data()
distToRidge = movedMesh.cell_arrays['implicit_distance']
isCloseToRidge = (distToRidge < 1000000.0)


earth.setRemeshClusters()
clusterMesh = movedMesh.extract_points(earth.isCluster)
clusterMesh = pv.PolyData(clusterMesh.points, clusterMesh.cells)
distToCluster = movedMesh.compute_implicit_distance(clusterMesh, inplace=True)
distToCluster = distToCluster.point_data_to_cell_data().cell_arrays['implicit_distance']

clusterPoints = movedEarthXYZ[earth.isCluster]
distToCluster = cKDTree(clusterPoints).query(movedEarthXYZ)[0]
notToCloseToCluster = (distToCluster > 400000)
movedMesh['distToCluster'] = distToCluster.astype(float)
notToCloseToCluster = movedMesh.point_data_to_cell_data().cell_arrays['distToCluster'].astype(bool)


isDiverging = (aspectArea > np.median(aspectArea) * 4.0)
isDiverging *= isCloseToRidge * notToCloseToCluster
movedMesh['isDiverging'] = isDiverging.astype(float)
movedMesh['isCloseToRidge'] = isCloseToRidge.astype(float)
movedMesh['distToCluster'] = distToCluster.astype(float)


from IPython import display
plotter = pv.Plotter()
plotter.add_mesh(movedMesh, scalars='distToCluster')
plotter.add_mesh(movedMesh.extract_all_edges(), color='white', opacity=0.2)
plotter.add_mesh(clusterMesh, color='purple')
#plotter.add_mesh(sizeMesh, color='white', opacity=0.2)
plotter.add_mesh(ridgeTube, color='cyan')
out = plotter.show()
display.display_html(out)


In [ ]:
#Create new earth object and move plates
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(pv.Sphere(radius=earth.earthRadius-200000))
plotter.add_mesh(movedMesh.extract_all_edges(), color='black')#, scalars=earth.heights)
plotter.add_mesh(movedMesh, scalars=earth.heights)
#plotter.add_points(movedEarthXYZ, color='b')
plotter.show()

In [ ]:
#Create cylinder of earth such that vertices are all equally spaced apart
#Overriding plates can then be identified as vertices being relatively closer to each other
def createEarthCylinder(earth):
    phiRes = earth.phiResolution
    thetaRes = earth.thetaResolution
    movedLonLat = earth.movedLonLat
    northToSoutDist = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    cylinderRadius = thetaRes * northToSoutDist / (np.pi * phiRes * 2)
    cylinderXYZ = EarthAssist.cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    return cylinderXYZ, thetaRes


def getMissingVertices(earth, subdivisions=3, medianMultiplier=1.3):
    
    movedLonLat = earth.movedLonLat
    exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
    movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)
    movedMesh['heights'] = earth.heights
    movedMesh['floorAge'] = earth.oceanFloorAge
    clusterPoints = movedEarthXYZ[earth.isCluster]
    
    #Create pyvista object of earth's cylindrical mesh
    cylinderXYZ, thetaRes = createEarthCylinder(earth)
    earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)

    #Calculate areas of each cell on the cylinder mesh
    cellAreas = earthCylinder.compute_cell_sizes().cell_arrays['Area']
    whereAreaIsLarge = np.argwhere(cellAreas > np.median(cellAreas) * medianMultiplier)
    
    
    #Missing vertices are identified by cells whose area is unusually large
    #So we create a mesh of just the unusually large cells
    blankAreas = movedMesh.extract_cells(whereAreaIsLarge)
    blankAreasXYZ = blankAreas.points
    
    #Do filtering here
    riftXYZ, riftLinePoints, riftSpeeds = earth.boundaries.getOceanicRifts() #Might need to use next iterations rifts
    dists = cKDTree(riftXYZ).query(blankAreasXYZ)[0]
    closeToRift = (dists < 1000000)

    distToCluster = cKDTree(clusterPoints).query(blankAreasXYZ)[0]
    notToCloseToCluster = (distToCluster > 100000)
    
    R, theta, phi = EarthAssist.cartesianToPolarCoords(blankAreasXYZ)
    isInOcean = (R - earth.earthRadius < -20000)
    combinedFilters = closeToRift * isInOcean * notToCloseToCluster
    
    
    blankAreas = blankAreas.extract_points(np.argwhere(combinedFilters))
    
    bordersXYZ = blankAreas.points
    borderHeights = blankAreas['heights']
    borderFloorAge = blankAreas['floorAge']

    #Convert the 'UnstructeredGrid' object to a 'PolyData' object, so we can subdivide the mesh
    blankAreas = pv.PolyData(blankAreas.points, blankAreas.cells)
    blankAreas = blankAreas.subdivide(subdivisions, 'linear')

    #The missing points are given by the points of the subdivided mesh
    return blankAreas, bordersXYZ, borderHeights, borderFloorAge




earth.setRemeshClusters()
#riftVertices, _, _ = getRiftVertices(earth)
blankAreas, bordersXYZ, borderHeights, borderFloorAge = getMissingVertices(earth)
riftVertices = blankAreas.points

movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
clusterPoints = movedEarthXYZ[earth.isCluster]

movedEarthMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)
movedEarthMesh['heights'] = earth.heights
boundaryMesh = earth.boundaries.getBoundaryMesh()

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(movedEarthMesh, scalars='heights')
plotter.add_mesh(movedEarthMesh.extract_all_edges(), color='white', opacity=0.1)
plotter.add_points(riftVertices, color='r')
plotter.add_points(clusterPoints, color='b')
plotter.add_mesh(boundaryMesh)
plotter.show()

Need to assign height and ocean floor age to riftVertices

In [ ]:
#Create new earth object and move plates
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])




earth.setRemeshClusters()
earth.setPlateData(earth.startTime - earth.deltaTime)
newBounds = earth.boundaries
newBoundMesh = newBounds.getBoundaryMesh()

newBoundsXYZ, boundLinePoints = [], []
for bound in newBounds.plateBoundaries:
    for i in range(bound.lineCentres.shape[0]):
        newBoundsXYZ.append(bound.lineCentres[i])
        boundLinePoints.append(bound.linePoints[i])
newBoundsXYZ = np.array(newBoundsXYZ)
boundLinePoints = np.array(boundLinePoints)

newRiftXYZ, newRiftLinePoints, newRiftSpeeds = newBounds.getOceanicRifts()
blankAreas, bordersXYZ, borderHeights, borderFloorAge = getMissingVertices(earth, subdivisions=5)
blankAreasXYZ = blankAreas.points

#distIds = cKDTree(newRiftXYZ).query(blankAreasXYZ)[1]
distIds = cKDTree(newBoundsXYZ).query(blankAreasXYZ)[1]
distIds[distIds >= newBoundsXYZ.shape[0]] = newBoundsXYZ.shape[0]-1
closestLinePoints = boundLinePoints[distIds]
distToRift = Boundaries.getDistsToLinesSeg(blankAreasXYZ, closestLinePoints)
distToRiftNorm = distToRift / np.max(distToRift)

distToBorder = cKDTree(bordersXYZ).query(blankAreasXYZ)[0]
distToBorderNorm = distToBorder / np.max(distToBorder)

#toCentreProportion = distToBorderNorm * (1 - distToRiftNorm)
#toCentreProportion = distToRiftNorm * (1 - distToBorderNorm)
toCentreProportion = distToBorder / (distToBorder + distToRift)
#toCentreProportion = distToRift / (distToBorder + distToRift)




movedEarthMesh = pv.PolyData(movedEarthXYZ * 0.99, earth.earthFaces)
movedEarthMesh['heights'] = earth.heights

plotter = pv.PlotterITK()
plotter.add_mesh(blankAreas, scalars=toCentreProportion**1)
plotter.add_mesh(movedEarthMesh, scalars='heights')
plotter.add_mesh(movedEarthMesh.extract_all_edges(), color='white', opacity=0.1)
plotter.add_points(bordersXYZ, color='r')
plotter.add_points(newRiftXYZ, color='y')
plotter.add_mesh(newBoundMesh)
plotter.show()

In [ ]:
#Create new earth object and move plates
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

'''
#Get aspectArea, a nice way of identifying diverging mesh areas
cellQuality = movedMesh.compute_cell_quality(quality_measure='area')
earthArea = cellQuality.cell_arrays['CellQuality']
cellQuality = movedMesh.compute_cell_quality(quality_measure='aspect_ratio')
aspectRatio = cellQuality.cell_arrays['CellQuality']
aspectArea = aspectRatio * earthArea
'''

plotter = pv.PlotterITK()
plotter.add_mesh(movedMesh)
plotter.show()

In [ ]:
#Create new earth object and move plates
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

#Get aspectArea, a nice way of identifying diverging mesh areas
cellQuality = movedMesh.compute_cell_quality(quality_measure='area')
earthArea = cellQuality.cell_arrays['CellQuality']
cellQuality = movedMesh.compute_cell_quality(quality_measure='aspect_ratio')
aspectRatio = cellQuality.cell_arrays['CellQuality']
aspectArea = aspectRatio * earthArea

#Need distance to oceanic rifts
earth.setPlateData(earth.startTime - earth.deltaTime)
newBounds = earth.boundaries

xyzCount = 0
newBoundsXYZ, lineConnectivity = [], []
for bound in newBounds.plateBoundaries:
    if bound.gpmlBoundType == 'gpml:MidOceanRidge':
        numOfPoints = bound.XYZ.shape[0]
        lineConnectivity.append(numOfPoints)
        lineID = np.arange(numOfPoints) + xyzCount
        for i in range(numOfPoints):
            newBoundsXYZ.append(bound.XYZ[i])
            lineConnectivity.append(lineID[i])
        xyzCount += numOfPoints

#Get distance to ridge
ridgeMesh = pv.PolyData(np.array(newBoundsXYZ), lines=lineConnectivity)
ridgeTube = ridgeMesh.tube(radius=10000, capping=False, n_sides=3)
movedMesh = movedMesh.compute_implicit_distance(ridgeTube, inplace=True).point_data_to_cell_data()
distToRidge = movedMesh.cell_arrays['implicit_distance']
isCloseToRidge = (distToRidge < 1000000.0)


earth.setRemeshClusters()
clusterMesh = movedMesh.extract_points(earth.isCluster)
clusterMesh = pv.PolyData(clusterMesh.points, clusterMesh.cells)
distToCluster = movedMesh.compute_implicit_distance(clusterMesh, inplace=True)
distToCluster = distToCluster.point_data_to_cell_data().cell_arrays['implicit_distance']

clusterPoints = movedEarthXYZ[earth.isCluster]
distToCluster = cKDTree(clusterPoints).query(movedEarthXYZ)[0]
notToCloseToCluster = (distToCluster > 400000)
movedMesh['distToCluster'] = distToCluster.astype(float)
notToCloseToCluster = movedMesh.point_data_to_cell_data().cell_arrays['distToCluster'].astype(bool)


isDiverging = (aspectArea > np.median(aspectArea) * 4.0)
isDiverging *= isCloseToRidge * notToCloseToCluster
movedMesh['isDiverging'] = isDiverging.astype(float)
movedMesh['isCloseToRidge'] = isCloseToRidge.astype(float)
movedMesh['distToCluster'] = distToCluster.astype(float)


from IPython import display
plotter = pv.PlotterITK()
plotter.add_mesh(movedMesh, scalars='isDiverging')
plotter.add_mesh(movedMesh.extract_all_edges(), color='white', opacity=0.2)
plotter.add_mesh(clusterMesh, color='purple')
plotter.add_mesh(sizeMesh, color='white', opacity=0.2)
plotter.add_mesh(ridgeTube, color='cyan')
out = plotter.show()
display.display_html(out)


In [ ]:
divergingMesh = movedMesh.extract_cells(np.argwhere(isDiverging))
divergingEdges = divergingMesh.extract_feature_edges(non_manifold_edges=False, feature_edges=False, manifold_edges=False)
#divergingEdges = divergingEdges.tube(radius=10000)
divergeSubdivided = pv.PolyData(divergingMesh.points, divergingMesh.cells).subdivide(3)



plotter = pv.PlotterITK()
plotter.add_mesh(pv.Sphere())
#plotter.add_mesh(divergingMesh)
#plotter.add_mesh(divergingEdges, color='r')
#plotter.add_mesh(divergeSubdivided.points, color = 'cyan')
#plotter.add_mesh(ridgeTube, color='y')
plotter.show(True)

In [ ]:
from IPython import display
import pyvista as pv

plotter = pv.Plotter()
#plotter.add_mesh(pv.Sphere())
plotter.add_mesh(divergingMesh)
#plotter.add_mesh(divergingEdges, color='r')
#plotter.add_mesh(divergeSubdivided.points, color = 'cyan')
#plotter.add_mesh(ridgeTube, color='y')
out = plotter.show()
display.display_html(out)


In [ ]:
plotter = pv.PlotterITK()
plotter.add_mesh(movedMesh)
plotter.show()

In [ ]:
#Create new earth object and move plates
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)
movePlates(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
movedMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

#Create pyvista object of earth's cylindrical mesh
cylinderXYZ, thetaRes = createEarthCylinder(earth)
earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)
earthCylinder = earthCylinder.compute_cell_quality(quality_measure='area')
cylinderArea = earthCylinder.cell_arrays['CellQuality']
earthCylinder = earthCylinder.compute_cell_quality(quality_measure='min_angle')
cylinderAngles = earthCylinder.cell_arrays['CellQuality']

cellQuality = movedMesh.compute_cell_quality(quality_measure='min_angle')
earthAngle = cellQuality.cell_arrays['CellQuality']
cellQuality = movedMesh.compute_cell_quality(quality_measure='area')
earthArea = cellQuality.cell_arrays['CellQuality']



cellQuality = movedMesh.compute_cell_quality(quality_measure='aspect_ratio')
testQuantity = cellQuality.cell_arrays['CellQuality']
testQuantity = testQuantity * earthArea


areaIsLarger = (earthArea > np.median(earthArea) * 1.2)
angleIsSmall = (earthAngle < np.median(earthAngle) * 0.7)
cylinderAreaIsLarger = (cylinderArea > np.median(cylinderArea) * 1.2)
cylinderAngleIsSmall = (cylinderAngles > np.median(cylinderAngles) * 0.9)

filt = cylinderAreaIsLarger + angleIsSmall


movedMesh['earthAngle'] = earthAngle.astype(float)
movedMesh['cylinderArea'] = cylinderArea.astype(float)
movedMesh['angleIsSmall'] = angleIsSmall.astype(float)
movedMesh['cylinderAreaIsLarger'] = cylinderAreaIsLarger.astype(float)
movedMesh['cylinderAngleIsSmall'] = cylinderAngleIsSmall.astype(float)
movedMesh['filt'] = filt.astype(float)
movedMesh['testQuantity'] = testQuantity.astype(float)

#print(movedMesh)
#print(cellQuality)

#sizeMesh = movedMesh.extract_all_edges().compute_cell_sizes()
#edgeLengths = sizeMesh.cell_arrays['Length']
#longEdges = sizeMesh.extract_cells(edgeLengths > np.mean(edgeLengths) * 1.4)

#Calculate areas of each cell on the cylinder mesh
#cellAreas = earthCylinder.compute_cell_sizes().cell_arrays['Area']
#whereAreaIsLarge = np.argwhere(cellAreas > np.median(cellAreas) * medianMultiplier)

plotter = pv.PlotterITK()
plotter.add_mesh(movedMesh, scalars='testQuantity')
plotter.add_mesh(sizeMesh, color='white', opacity=0.3)
plotter.show()

In [ ]:
plane = pv.Plane()
edges = plane.extract_feature_edges()

plotter = pv.PlotterITK()
plotter.add_mesh(blankAreas.extract_feature_edges())
plotter.show()


In [ ]:
plane = pv.Plane()
line = pv.Line((-0.5, -0.2, 0), (0.5, 0.2, 0))
tube = line.tube(radius=0.005, capping=False, n_sides=3)

#Old method for distance calculations
distToLine = cKDTree(line.points).query(plane.points)[0]

plane.compute_implicit_distance(tube, inplace=True)


plotter = pv.PlotterITK()
plotter.add_mesh(plane, scalars='implicit_distance')
plotter.add_mesh(plane.extract_all_edges(), color='white', opacity=0.2)
plotter.add_mesh(tube)
plotter.add_mesh(tube.extract_all_edges(), color='white', opacity=0.2)
plotter.show()

In [ ]:
'''
#Get missing vertices at rift boundaries
def getRiftVertices(earth):
    isCluster = earth.isCluster
    blankAreas, bordersXYZ, borderHeights, borderFloorAge = getMissingVertices(earth)
    
    movedLonLat = earth.movedLonLat
    exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
    movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    clusterPoints = movedEarthXYZ[isCluster]
    
    riftXYZ, riftLinePoints, riftSpeeds = earth.boundaries.getOceanicRifts() #Might need to use next iterations rifts
    dists = cKDTree(riftXYZ).query(blankAreas)[0]
    closeToRift = (dists < 1000000)

    distToCluster = cKDTree(clusterPoints).query(blankAreas)[0]
    notToCloseToCluster = (distToCluster > 100000)
    
    R, theta, phi = EarthAssist.cartesianToPolarCoords(blankAreas)
    isInOcean = (R - earth.earthRadius < -20000)
    
    combinedFilters = closeToRift * isInOcean * notToCloseToCluster
    blankAreas = blankAreas[combinedFilters] 
    #borderHeights = borderHeights[combinedFilters] 
    #borderFloorAge = borderFloorAge[combinedFilters]
    
    return blankAreas, borderHeights, borderFloorAge
'''

In [ ]:
from EarthsAssistant import EarthAssist

blankAreas = getMissingVertices(earth)
R, theta, phi = EarthAssist.cartesianToPolarCoords(blankAreas)
plt.plot(earth.earthRadius - R)


In [ ]:
#Create cylinder of earth such that vertices are all equally spaced apart
#Overriding plates can then be identified as vertices being relatively closer to each other
def createEarthCylinder(earth):
    phiRes = earth.phiResolution
    thetaRes = earth.thetaResolution
    movedLonLat = earth.movedLonLat
    northToSoutDist = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    cylinderRadius = thetaRes * northToSoutDist / (np.pi * phiRes * 2)
    cylinderXYZ = EarthAssist.cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    return cylinderXYZ, thetaRes

#Get vertices to fill in missing vertices for when plates diverge
def getMissingVertices(earth, subdivisions=3):
    
    #Create pyvista object of earth's cylindrical mesh
    cylinderXYZ, thetaRes = createEarthCylinder(earth)
    earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)
    print(earthCylinder)
    
    #Calculate areas of each cell on the cylinder mesh
    cellAreas = earthCylinder.compute_cell_sizes().cell_arrays['Area']
    whereAreaIsLarge = np.argwhere(cellAreas > 1)
    print(cellAreas.shape)
    
    #Missing vertices are identified by cells whose area is unusually large
    #So we create a mesh of just the unusually large cells
    
    #Need to get whereIsConverging... Use get cell centres?
    blankAreas = movedEarthMesh.extract_cells(whereAreaIsLarge)
    
    #Convert the 'UnstructeredGrid' object to a 'PolyData' object, so we can subdivide the mesh
    blankAreas = pv.PolyData(blankAreas.points, blankAreas.cells)
    
    #The missing points are given by the points of the subdivided mesh
    return blankAreas.subdivide(subdivisions, 'linear').points
    
movedEarthMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)
blankAreas = getMissingVertices(earth)

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(movedEarthMesh, scalars=earth.heights)
plotter.add_mesh(movedEarthMesh.extract_all_edges(), color='black')
plotter.add_points(blankAreas, color='r')
plotter.show()

In [ ]:
riftXYZ, riftLinePoints, riftSpeeds = earth.boundaries.getOceanicRifts()

In [ ]:
def getMissingVertices(earth, subdivisions=3):
    #Create pyvista object of earth's cylindrical mesh
    cylinderXYZ, thetaRes = createEarthCylinder(earth)
    earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)

    #Calculate areas of each cell on the cylinder mesh
    cellAreas = earthCylinder.compute_cell_sizes().cell_arrays['Area']
    whereAreaIsLarge = np.argwhere(cellAreas > 1)


subdivisions = 3

earth.setRemeshClusters()
isCluster = earth.isCluster
clusterPoints = movedEarthXYZ[isCluster]
clusterMesh = movedEarthMesh.extract_points(np.argwhere(isCluster))


#Create pyvista object of earth's cylindrical mesh
cylinderXYZ, thetaRes = createEarthCylinder(earth)
earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)

#Calculate areas of each cell on the cylinder mesh
cellAreas = earthCylinder.compute_cell_sizes().cell_arrays['Area']
whereAreaIsLarge = np.argwhere(cellAreas > 1)

#Missing vertices are identified by cells whose area is unusually large
#So we create a mesh of just the unusually large cells

#Need to get whereIsConverging... Use get cell centres?
blankAreas = movedEarthMesh.extract_cells(whereAreaIsLarge)

#Convert the 'UnstructeredGrid' object to a 'PolyData' object, so we can subdivide the mesh
blankAreas = pv.PolyData(blankAreas.points, blankAreas.cells)

#The missing points are given by the points of the subdivided mesh
blankAreas = blankAreas.subdivide(subdivisions, 'linear').points


riftXYZ, riftLinePoints, riftSpeeds = earth.boundaries.getOceanicRifts() #Might need to use next iterations rifts
dists = cKDTree(riftXYZ).query(blankAreas)[0]
closeToRift = (dists < 1000000)

distToCluster = cKDTree(clusterPoints).query(blankAreas)[0]
notToCloseToCluster = (distToCluster > 100000)
    
movedEarthMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)
boundaryMesh = earth.boundaries.getBoundaryMesh()

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(movedEarthMesh, scalars=earth.heights)
plotter.add_mesh(movedEarthMesh.extract_all_edges(), color='white', opacity=0.1)
plotter.add_points(blankAreas[closeToRift*notToCloseToCluster], color='r')
plotter.add_points(clusterPoints, color='b')
plotter.add_mesh(boundaryMesh)
plotter.show()

In [ ]:
plt.plot(np.log(cellAreas))
print(np.mean(cellAreas))
print(np.median(cellAreas))

Need to remove vertices where plates converge.

In [ ]:
earth.setRemeshClusters()
isCluster = earth.isCluster
clusterMesh = movedEarthMesh.extract_points(np.argwhere(isCluster))


#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(clusterMesh)
plotter.show()

In [ ]:
subdivisions = 3

#Create pyvista object of earth's cylindrical mesh
cylinderXYZ, thetaRes = createEarthCylinder(earth)
earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)


#Calculate areas of each cell on the cylinder mesh
cellAreas = earthCylinder.compute_cell_sizes().cell_arrays['Area']
whereAreaIsLarge = np.argwhere(cellAreas > 1)

#Missing vertices are identified by cells whose area is unusually large
#So we create a mesh of just the unusually large cells

#Need to get whereIsConverging... Use get cell centres?
blankAreas = movedEarthMesh.extract_cells(whereAreaIsLarge)

#Convert the 'UnstructeredGrid' object to a 'PolyData' object, so we can subdivide the mesh
blankAreas = pv.PolyData(blankAreas.points, blankAreas.cells)

#The missing points are given by the points of the subdivided mesh
blankAreas = blankAreas.subdivide(subdivisions, 'linear').points
    
movedEarthMesh = pv.PolyData(movedEarthXYZ, earth.earthFaces)

boundaryMesh = earth.boundaries.getBoundaryMesh()

#Visualize the results
plotter = pv.PlotterITK()
plotter.add_mesh(movedEarthMesh, scalars=earth.heights)
plotter.add_mesh(movedEarthMesh.extract_all_edges(), color='white', opacity=0.1)
plotter.add_points(blankAreas, color='r')
plotter.add_points(clusterMesh, color='b')
plotter.add_mesh(boundaryMesh)
plotter.show()

In [ ]:
def getInterpolationCoordinates():
    divergeVertices = getMissingVertices(earth)
    
#Function based on interpolateScalar() from earth's remesh algorithm
def interpolateScalar(earth, scalar):
    scalarForRemesh = earth.prepareScalarsForRemesh(scalar)
    movedLonLat = earth.movedLonLat
    newScalar = griddata(movedLonLat, scalarForRemesh, earth.lonLat)
    whereNAN = np.argwhere(np.isnan(newScalar))
    newScalar[whereNAN] = griddata(movedLonLat, scalarForRemesh, earth.lonLat[whereNAN], method='nearest')
    return newScalar

In [ ]:
#Algorithm for moving plates and without remeshing the sphere
def movePlates(earth):
    movedEarthXYZ = earth.movePlates(earth.plateIds, earth.rotations)
    movedLonLat = EarthAssist.cartesianToPolarCoords(movedEarthXYZ)
    earth.movedLonLat = np.stack((movedLonLat[1], movedLonLat[2]), axis=1)

#Create cylinder of earth such that vertices are all equally spaced apart
#Overriding plates can then be identified as vertices being relatively closer to each other
def createEarthCylinder(earth):
    phiRes = earth.phiResolution
    thetaRes = earth.thetaResolution
    movedLonLat = earth.movedLonLat
    northToSoutDist = np.max(movedLonLat[:, 1]) - np.min(movedLonLat[:, 1])
    cylinderRadius = thetaRes * northToSoutDist / (np.pi * phiRes * 2)
    cylinderXYZ = EarthAssist.cylindricalToCartesian(cylinderRadius, movedLonLat[:, 0], movedLonLat[:, 1])
    return cylinderXYZ, thetaRes   
    
#Create new earth object
earth = Earth(startTime=60, endTime=0, deltaTime=10, minClusterSize=2)

#Move plates and set remesh clusters
movePlates(earth)
cylinderXYZ, thetaRes = createEarthCylinder(earth)

#Get data for visualizations
movedLonLat = earth.movedLonLat
exageratedRadius = earth.heightHistory[-1] * earth.heightAmplificationFactor + earth.earthRadius
movedEarthXYZ = EarthAssist.polarToCartesian(exageratedRadius, movedLonLat[:, 0], movedLonLat[:, 1])

earthCylinder = pv.PolyData(cylinderXYZ, earth.earthFaces)

cellSized = earthCylinder.compute_cell_sizes()
cellAreas = cellSized.cell_arrays['Area']
cellsWithLargeArea = cellAreas > 1
largeCellCylinder = earthCylinder.extract_cells(np.argwhere(cellsWithLargeArea))

#mesh = largeCellCylinder.triangulate()
mesh = pv.PolyData(largeCellCylinder.points, largeCellCylinder.cells)

print(mesh)
print(pv.Sphere())

#largeCellCylinder = largeCellCylinder.triangulate()


#for i in largeCellCylinder.cells[:100]:
#    print(i)

#Visualize the results
plotter = pv.PlotterITK()
#plotter.add_mesh(pv.Sphere(radius=earth.earthRadius-200000))
#plotter.add_mesh(earthCylinder, scalars=cellsWithLargeArea.astype(float))
plotter.add_mesh(mesh.subdivide(3, 'linear'), color='b')
#plotter.add_mesh(cellSizes, scalars=cellSizes.cell_arrays['Area'])
plotter.show()


In [ ]:
print((23 - 9)/4)
print(9 + 3.5 * 2)

In [ ]:
earth.animate(lookAtLonLat=[120, 0])

In [ ]:
earth.showEarth()

In [ ]:
crash Python here

In [ ]:
earth = Earth(startTime = 60, endTime = 0, deltaTime = 10)
earth.doSimulationStep(earth.startTime)
earth.setPlateData(earth.startTime)
bounds = earth.boundaries

earthMesh = earth.getEarthMesh()
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh)
plotter.show()

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from EarthsAssistant import EarthAssist

#Want speeds to nearest diverging boundary
def getSpeedsToDivergence(divXYZ, sphereXYZ, linePoints):
    distIds = cKDTree(divXYZ).query(sphereXYZ, k=50)[1]
    distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
    closestLinePoints = linePoints[distIds]
    distToBound = Boundaries.getDistsToLinesSeg(sphereXYZ, closestLinePoints)

#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(bounds):
    divXYZ, divLinePoints, divSpeeds = [], [], []
    for bound in bounds.plateBoundaries:
        if bound.boundType == 2:
            for i in range(bound.lineCentres.shape[0]):
                if bound.collisionSpeed[i] < 0:
                    divXYZ.append(bound.lineCentres[i])
                    divLinePoints.append(bound.linePoints[i])
                    divSpeeds.append(bound.collisionSpeed[i])
    return np.array(divXYZ), np.array(divLinePoints), np.array(divSpeeds)

#Create new earth object
earth = Earth(
    startTime = 5,
    endTime = 0,
    deltaTime = 1)
earthMesh = earth.getEarthMesh()
earth.setPlateData(earth.startTime)
bounds = earth.boundaries

#Run the getUplifts function so we can get it's speed transfer (and avoid having to calculate it twice)
bounds.getUplifts()
speedTransfer = bounds.speedTransfer

#Get distance to diverging plate boundaries
#distToDivs = bounds.setDistanceToDivergence()

divXYZ, divLinePoints, divSpeeds = getDivergingBoundaries(bounds)

distIds = cKDTree(divXYZ).query(earth.sphereXYZ, k=10)[1]
distIds[distIds >= divXYZ.shape[0]] = divXYZ.shape[0]-1
closestLinePoints = divLinePoints[distIds]
distToBound = Boundaries.getDistsToLinesSeg(earth.sphereXYZ, closestLinePoints[:, 0])

divSpds = divSpeeds[distIds]
divSpds = np.mean(divSpds, axis=1)
#print(np.mean(divSpds, axis=1).shape)

In [ ]:
#gaussDeleteThis = EarthAssist.gaussian(distToBound/4000000)**0.5
contour = earthMesh.contour([0], scalars='heights')
boundaryMesh = bounds.getBoundaryMesh()


age = - distToBound / (divSpds + 10)
heights= earthMesh['heights']
heights[heights>0] = 0
heights = (-heights)**0.5
earthMesh['newHeights'] = heights


#cutOff = 200
#age[age>cutOff] = cutOff
age[earth.heights>0] = np.max(age)


#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars=age**0.5)
plotter.add_mesh(boundaryMesh)
plotter.add_mesh(contour, color="peru", opacity=1.)
plotter.show()

In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')

import numpy as np
import pyvista as pv
from TectonicEarth import Earth
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from GosplManager import GosplManager
from PlateBoundaries import Boundaries
from EarthsAssistant import EarthAssist


#Get coordinates and line points of all diverging plate boundary locations
def getDivergingBoundaries(bounds):
    divXYZ, divLinePoints, divSpeeds = [], [], []
    for bound in bounds.plateBoundaries:
        for i in range(bound.lineCentres.shape[0]):
            if bound.collisionSpeed[i] < 0:
                divXYZ.append(bound.lineCentres[i])
                divLinePoints.append(bound.linePoints[i])
                divSpeeds.append(bound.collisionSpeed[i])
    return np.array(divXYZ), np.array(divLinePoints), np.array(divSpeeds)

#Create new earth object
earth = Earth(
    startTime = 5,
    endTime = 0,
    deltaTime = 1)

#Create mesh objects
earthMesh = earth.getEarthMesh()
earth.setPlateData(earth.startTime)



speedTransfer, distTransfer = earth.boundaries.getTransfers()
uplift = earth.boundaries.getUplifts()
print(distTransfer)



#Show results
plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh, scalars=uplift)
plotter.show()

In [ ]:
#Create new earth object
earth = Earth(
    startTime = 30,
    endTime = 0,
    deltaTime = 1)

earth.runTectonicSimulation()
earth.showEarth()

In [ ]:
earth.animate()